# Ensemble Learning

## Initial Imports

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
df = pd.read_csv('LoanStats_2019Q1.csv')

# Preview the data
df.head()

,loan_amnt,int_rate,installment,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,dti,...,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag
0,10500.0,0.1719,375.35,RENT,66000.0,Source Verified,Mar-2019,low_risk,n,27.24,...,85.7,100.0,0.0,0.0,65687.0,38199.0,2000.0,61987.0,N,N
1,25000.0,0.2000,929.09,MORTGAGE,105000.0,Verified,Mar-2019,low_risk,n,20.23,...,91.2,50.0,1.0,0.0,271427.0,60641.0,41200.0,49197.0,N,N
2,20000.0,0.2000,529.88,MORTGAGE,56000.0,Verified,Mar-2019,low_risk,n,24.26,...,66.7,50.0,0.0,0.0,60644.0,45684.0,7500.0,43144.0,N,N
3,10000.0,0.1640,353.55,RENT,92000.0,Verified,Mar-2019,low_risk,n,31.44,...,100.0,50.0,1.0,0.0,99506.0,68784.0,19700.0,76506.0,N,N
4,22000.0,0.1474,520.39,MORTGAGE,52000.0,Not Verified,Mar-2019,low_risk,n,18.76,...,100.0,0.0,0.0,0.0,219750.0,25919.0,27600.0,20000.0,N,N


In [5]:
#inspect what needs to be cleaned
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68817 entries, 0 to 68816
Data columns (total 86 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   loan_amnt                   68817 non-null  float64
 1   int_rate                    68817 non-null  float64
 2   installment                 68817 non-null  float64
 3   home_ownership              68817 non-null  object 
 4   annual_inc                  68817 non-null  float64
 5   verification_status         68817 non-null  object 
 6   issue_d                     68817 non-null  object 
 7   loan_status                 68817 non-null  object 
 8   pymnt_plan                  68817 non-null  object 
 9   dti                         68817 non-null  float64
 10  delinq_2yrs                 68817 non-null  float64
 11  inq_last_6mths              68817 non-null  float64
 12  open_acc                    68817 non-null  float64
 13  pub_rec                     688

In [6]:
#data cleaning


## Split the Data into Training and Testing

In [7]:
# Create our features
X = df.drop(columns='loan_status')
X = pd.get_dummies(X)
X.head()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
0,10500.0,0.1719,375.35,66000.0,27.24,0.0,0.0,8.0,0.0,1609.0,...,1,1,0,1,0,1,1,0,1,1
1,25000.0,0.2000,929.09,105000.0,20.23,0.0,0.0,17.0,1.0,18368.0,...,1,1,0,1,0,1,1,0,1,1
2,20000.0,0.2000,529.88,56000.0,24.26,0.0,0.0,8.0,0.0,13247.0,...,1,1,0,1,0,1,1,0,1,1
3,10000.0,0.1640,353.55,92000.0,31.44,0.0,1.0,10.0,1.0,17996.0,...,1,1,0,1,0,1,1,0,1,1
4,22000.0,0.1474,520.39,52000.0,18.76,0.0,1.0,14.0,0.0,9091.0,...,1,1,0,1,0,1,1,0,1,1


In [8]:
# Create our target
y = df['loan_status']
y.head()

0    low_risk
1    low_risk
2    low_risk
3    low_risk
4    low_risk
Name: loan_status, dtype: object

In [9]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.000000,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.0
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,0.177238,1.0,0.123879,0.876121,0.383161,0.616839,0.860340,0.139660,1.0,1.0
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.381873,0.0,0.329446,0.329446,0.486161,0.486161,0.346637,0.346637,0.0,0.0
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [10]:
# Check the balance of our target values
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [11]:
# Split the X and y into X_train, X_test, y_train, y_test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)
X_train.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,51612.000000,51612.00000,51612.000000,5.161200e+04,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,...,51612.000000,51612.0,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.000000,51612.0,51612.0
mean,16657.813590,0.12766,480.095682,8.770656e+04,21.868028,0.214873,0.496280,12.579110,0.126405,17632.221654,...,0.177866,1.0,0.124118,0.875882,0.383729,0.616271,0.859103,0.140897,1.0,1.0
std,10246.336374,0.04801,287.144912,1.039301e+05,20.981640,0.705290,0.754984,6.026174,0.337687,21792.011481,...,0.382403,0.0,0.329720,0.329720,0.486298,0.486298,0.347919,0.347919,0.0,0.0
min,1000.000000,0.06000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9068.750000,0.08810,265.730000,5.000000e+04,13.900000,0.000000,0.000000,8.000000,0.000000,6291.500000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.11800,404.600000,7.300000e+04,19.770000,0.000000,0.000000,11.000000,0.000000,12067.500000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.15570,647.717500,1.040000e+05,26.722500,0.000000,1.000000,16.000000,0.000000,21763.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.30840,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,69.000000,4.000000,587191.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [12]:
X_test.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,issue_d_Mar-2019,pymnt_plan_n,initial_list_status_f,initial_list_status_w,next_pymnt_d_Apr-2019,next_pymnt_d_May-2019,application_type_Individual,application_type_Joint App,hardship_flag_N,debt_settlement_flag_N
count,17205.000000,17205.000000,17205.000000,1.720500e+04,17205.000000,17205.000000,17205.000000,17205.000000,17205.000000,17205.000000,...,17205.000000,17205.0,17205.000000,17205.000000,17205.000000,17205.000000,17205.000000,17205.000000,17205.0,17205.0
mean,16736.934031,0.127892,482.324309,8.973505e+04,21.508542,0.226446,0.501947,12.612031,0.124906,17519.911247,...,0.175356,1.0,0.123162,0.876838,0.381459,0.618541,0.864051,0.135949,1.0,1.0
std,10369.896629,0.048489,290.799455,1.450105e+05,17.643096,0.756195,0.767464,6.013051,0.334119,21967.374471,...,0.380282,0.0,0.328632,0.328632,0.485759,0.485759,0.342744,0.342744,0.0,0.0
min,1000.000000,0.064600,30.890000,5.000000e+02,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.0,1.0
25%,9000.000000,0.088100,264.940000,5.000000e+04,13.860000,0.000000,0.000000,8.000000,0.000000,6297.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.0,1.0
50%,15000.000000,0.118000,404.220000,7.300000e+04,19.710000,0.000000,0.000000,11.000000,0.000000,12071.000000,...,0.000000,1.0,0.000000,1.000000,0.000000,1.000000,1.000000,0.000000,1.0,1.0
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.450000,0.000000,1.000000,16.000000,0.000000,21636.000000,...,0.000000,1.0,0.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0
max,40000.000000,0.308400,1674.060000,8.779200e+06,999.000000,15.000000,5.000000,72.000000,4.000000,562743.000000,...,1.000000,1.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.0


In [13]:
y_train.describe()

count        51612
unique           2
top       low_risk
freq         51352
Name: loan_status, dtype: object

In [14]:
y_test.describe()

count        17205
unique           2
top       low_risk
freq         17118
Name: loan_status, dtype: object

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [27]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler

StandardScaler()

In [31]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
X_scaler = scaler.fit(X_train)
X_scaler

StandardScaler()

In [32]:
# Scale the training and testing data
X_train_scaled = X_scaler.transform(X_train)
X_train_scaled

array([[-0.64978133, -0.30327574, -0.52669276, ..., -0.40497582,
         0.        ,  0.        ],
       [ 0.42378352, -0.42616892,  0.71269346, ..., -0.40497582,
         0.        ,  0.        ],
       [ 1.79013878, -0.82400957,  2.19335392, ..., -0.40497582,
         0.        ,  0.        ],
       ...,
       [ 0.81417074, -0.36368086,  1.17925683, ..., -0.40497582,
         0.        ,  0.        ],
       [-0.25939411, -0.42616892, -0.08220905, ..., -0.40497582,
         0.        ,  0.        ],
       [-0.16179731, -1.196855  , -0.05849257, ..., -0.40497582,
         0.        ,  0.        ]])

In [33]:
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[-1.23536216, -1.196855  , -1.24170452, ...,  2.46928324,
         0.        ,  0.        ],
       [ 0.22858991,  0.06540381, -0.16370179, ...,  2.46928324,
         0.        ,  0.        ],
       [-0.45458772,  0.35493183, -0.68932999, ..., -0.40497582,
         0.        ,  0.        ],
       ...,
       [-1.13776535,  0.35493183, -1.07283345, ..., -0.40497582,
         0.        ,  0.        ],
       [ 1.30215476, -0.95315156,  0.45599132, ..., -0.40497582,
         0.        ,  0.        ],
       [-0.16179731,  0.46949327,  0.13942335, ..., -0.40497582,
         0.        ,  0.        ]])

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [34]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf = BalancedRandomForestClassifier(n_estimators=100, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(random_state=1)

In [41]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7877672625306695

In [42]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

array([[   58,    29],
       [ 1560, 15558]], dtype=int64)

In [43]:
# Print the imbalanced classification report
y_pred_rf = brf.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.67      0.91      0.07      0.78      0.59        87
   low_risk       1.00      0.91      0.67      0.95      0.78      0.62     17118

avg / total       0.99      0.91      0.67      0.95      0.78      0.62     17205



In [35]:
# List the features sorted in descending order by feature importance
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.07376667607601396, 'total_rec_prncp'),
 (0.06390324452717588, 'total_rec_int'),
 (0.06073336071656837, 'total_pymnt_inv'),
 (0.05811195697921674, 'total_pymnt'),
 (0.04951778391272079, 'last_pymnt_amnt'),
 (0.02458051789018817, 'int_rate'),
 (0.020398879691407974, 'out_prncp'),
 (0.018625883307901298, 'dti'),
 (0.018378884967316288, 'max_bal_bc'),
 (0.017480030880564042, 'issue_d_Jan-2019'),
 (0.01698703048268034, 'installment'),
 (0.01661321625916749, 'annual_inc'),
 (0.01623198470200134, 'bc_util'),
 (0.015821923028819504, 'mths_since_recent_inq'),
 (0.015385686591272854, 'total_bc_limit'),
 (0.015337070167127763, 'out_prncp_inv'),
 (0.01506649550412446, 'avg_cur_bal'),
 (0.014926455495755147, 'issue_d_Mar-2019'),
 (0.014924489307566175, 'revol_bal'),
 (0.014583210727445232, 'mo_sin_old_rev_tl_op'),
 (0.01428231366275729, 'total_bal_ex_mort'),
 (0.014238473969302518, 'tot_cur_bal'),
 (0.014067369111282934, 'mo_sin_old_il_acct'),
 (0.01351439379612201, 'total_bal_il'),
 (0.0133438

### Easy Ensemble Classifier

In [37]:
# Train the Classifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [38]:
# Calculated the balanced accuracy score
y_pred_eec = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred_eec)

0.925427358175101

In [39]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred_eec)

array([[   79,     8],
       [  979, 16139]], dtype=int64)

In [40]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred_eec))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.07      0.91      0.94      0.14      0.93      0.85        87
   low_risk       1.00      0.94      0.91      0.97      0.93      0.86     17118

avg / total       0.99      0.94      0.91      0.97      0.93      0.86     17205



### Final Questions

1. Which model had the best balanced accuracy score?

    The Easy Ensemble Classifier model had the best balanced accuracy score with the most correct calls, with a score of 0.9254 versus score of 0.7877.

2. Which model had the best recall score?

    The Easy Ensemble Classifier model had the best recall score (actual postitive samples), with scores of 0.91 for high risk and 0.94 for low risk versus recall scores of 0.67 and 0.91 for Balanced Random Forest Classifiers. This means that the Easy Ensemble Classifier had a higher proportion of actually positive samples.

3. Which model had the best geometric mean score?

    The Easy Ensemble Classifier model with both high risk and low risk of 0.93 versus 0.78.

4. What are the top three features?

    The top three features of the Balanced Random Forest Classifier model are as follows: total_rec_prncp : 0.0737, total_pymnt : 0.0639, total_pymnt_inv : 0.0607
    